## Collateral Asset Selection and Scope Assumptions

The collateral portfolio used in this analysis is designed to reflect a realistic and simplified Lombard lending setup for a high-net-worth private banking client. The focus of the model is on collateral-driven risk dynamics rather than on asset selection or portfolio optimisation.

For this reason, the collateral universe is intentionally limited to a small number of highly liquid and broadly representative asset classes, which are commonly accepted as eligible collateral in private banking Lombard facilities.

### Equity Component

The equity component of the collateral portfolio is represented by a broad market equity ETF tracking the S&P 500 index (Vanguard S&P 500 UCITS ETF). This instrument is used as a proxy for a diversified equity allocation, avoiding concentration risk associated with single stocks or sector-specific exposures.

Equities typically represent the primary source of volatility within a Lombard collateral portfolio and therefore play a key role in driving loan-to-value (LTV) dynamics under market stress.

### Bond Component

The fixed income component is represented by an investment grade bond ETF with intermediate duration (iShares Core Global / Euro Aggregate Bond UCITS ETF). This asset class reflects a typical core bond allocation used for income generation and capital preservation.

Compared to equities, investment grade bonds exhibit lower volatility and are therefore subject to more conservative price fluctuations, providing partial stabilisation of collateral value during equity market downturns.

### Cash Component

A cash or money market allocation is included as a liquidity buffer. Rather than modelling specific money market instruments or short-term securities, cash is treated as a stable, cash-equivalent position with negligible price volatility.

This approach reflects how cash and money market holdings are effectively treated in collateral risk systems, where the focus is on available value rather than on instrument-level maturity or reinvestment mechanics.

### Currency Assumptions

All assets are considered in a single currency. Foreign exchange risk is intentionally excluded from the analysis in order to isolate and focus on collateral price risk. This simplification avoids introducing additional layers of complexity that are not central to the objectives of the model.

### Scope and Modelling Choices

The model is designed as a historical, scenario-based risk analysis rather than a full balance sheet or regulatory capital framework. As such, the following elements are explicitly excluded:

- foreign exchange risk and currency conversion effects  
- loan pricing, interest accrual and repayment schedules  
- borrower creditworthiness and income considerations  
- regulatory capital or accounting treatment  

The primary objective is to analyse how changes in market prices of pledged collateral translate into LTV movements, margin calls and liquidation events over time under realistic market conditions.


In [ ]:
import pandas as pd
import numpy as np

vanguard = pd.read_csv(r"Vanguard_ETF_SP500_EUR.csv")
vanguard['Date'] = pd.to_datetime(vanguard['Date'])
bond = pd.read_csv(r"Bond_Investm_grade_EUR.csv")
bond['Date'] = pd.to_datetime(bond['Date'])


In [156]:
vanguard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1532 entries, 0 to 1531
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      1532 non-null   datetime64[ns]
 1   Price     1532 non-null   float64       
 2   Open      1532 non-null   float64       
 3   High      1532 non-null   float64       
 4   Low       1532 non-null   float64       
 5   Vol.      1531 non-null   object        
 6   Change %  1532 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 83.9+ KB


In [157]:
bond.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1532 entries, 0 to 1531
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      1532 non-null   datetime64[ns]
 1   Price     1532 non-null   float64       
 2   Open      1532 non-null   float64       
 3   High      1532 non-null   float64       
 4   Low       1532 non-null   float64       
 5   Vol.      1531 non-null   object        
 6   Change %  1532 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 83.9+ KB


In [158]:
vanguard = vanguard.set_index('Date')
vanguard = vanguard.sort_values(by='Date')
vanguard.isna().sum()

Price       0
Open        0
High        0
Low         0
Vol.        1
Change %    0
dtype: int64

In [159]:
vanguard[vanguard.isna().values]

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2024-09-16,95.68,95.89,95.98,95.58,NaN,-0.47%


In [160]:
bond = bond.set_index('Date')
bond = bond.sort_values(by='Date')
bond.isna().sum()

Price       0
Open        0
High        0
Low         0
Vol.        1
Change %    0
dtype: int64

In [161]:
bond[bond.isna().values]

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2024-09-16,4.9,4.9,4.9,4.89,NaN,0.00%


In [162]:
vanguard.duplicated().sum()

np.int64(0)

In [163]:
bond.duplicated().sum()

np.int64(0)

### Cleaning Data

In [164]:
def cleaning_and_merging(vanguard_df, bond_df):
    vanguard_df = vanguard_df.copy()
    bond_df = bond_df.copy()
    
    ptf_df = vanguard_df.join(bond_df, how='left', lsuffix = '_Equity', rsuffix = '_Bond' ).dropna()
    return ptf_df

ptf_df = cleaning_and_merging(vanguard, bond)
price_matrix = ptf_df.loc[: ,[ 'Price_Equity', 'Price_Bond']]
price_matrix ['Cash'] = int(500000 * 0.1)
price_matrix

,Price_Equity,Price_Bond,Cash
Date,,,
2020-01-02,55.02,5.19,50000
2020-01-03,54.97,5.20,50000
2020-01-06,54.85,5.20,50000
2020-01-07,55.29,5.20,50000
2020-01-08,55.53,5.19,50000
...,...,...,...
2026-01-05,111.86,4.93,50000
2026-01-06,112.14,4.92,50000
2026-01-07,112.68,4.94,50000


### Sanity check on data: 

Checking missing and duplicate values

In [165]:
# check NaN
na_counts = price_matrix.isna().sum()
assert na_counts.sum() == 0, f"Found NaNs:\n{na_counts[na_counts>0]}"
# check unique date
assert price_matrix.index.is_unique
# check positive price
assert (price_matrix['Price_Equity'] > 0).all()
assert (price_matrix['Price_Bond'] > 0).all()